In [ ]:
#liberary
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import numpy as np
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import glob
import os

In [ ]:
def log_in(Web,person):
    bookmark = {}
    ID_password={}
    try:
        url=bookmark[Web]
    except KeyError :
        print("please fill in correct web:'ibuy','ERP_PM','ERP_PC','ISDP','ECOR'")
    try:
        ID=ID_password[person][0]
        password=ID_password[person][1]
    except KeyError :
        print("please fill in correct name:'Yifan','Yuqi','Wenji','Junjun','Mengmeng','Yizhe'")    
    driver.set_page_load_timeout(10)
    driver.get(url)
    driver.maximize_window()
    time.sleep(2)
    driver.find_element_by_name("uid").send_keys(ID)
    driver.find_element_by_name("password").send_keys(password)
    driver.find_element_by_name("password").send_keys(Keys.ENTER)

In [ ]:
path =r'D:\02.PO_cancellation\Request'
allFiles = glob.glob(path+ "/*.xlsx")
Cancel_table= pd.read_excel(allFiles[0], sheet_name='Sheet1',dtype=str) 

Column Name-setting manually

In [ ]:
PO_NO='PO NO.'
DU_ID='Site Code'
Item_code='Item'
Item_des='PO Line Description'

Index setting - Cancel by item_code

In [ ]:
Cancel_table.set_index(keys=['PO NO.','Site Code','Item'],inplace=True)
Cancel_table

Index setting - Cancel by item_Description

In [ ]:
Cancel_table.set_index(keys=['PO NO.','Site ID','PO Line Description'],inplace=True)
Cancel_table

Choose Cancel reason and write down remark

In [ ]:
Cancel_select="Canceled By Huawei"

In [ ]:
Cancel_select="Long Term PO"

In [ ]:
Cancel_select="Canceled By Customer"

In [ ]:
Cancel_select="Duplicate PO"

In [ ]:
Cancel_select="Canceled By Supplier"

In [ ]:
Cancel_reason='Priority 1'

remove PO had been cancelled 

In [ ]:
PO_list=list(set(list(Cancel_table.index.get_level_values('PO NO.'))))
PO_list

In [ ]:
Check_table= pd.read_excel(r'D:\02.PO_cancellation\Process\Check_process.xlsx', sheet_name='Sheet1')
Check_table

In [ ]:
for PO in list(set(list(Check_table['PO']))):
    PO_list.remove(PO)
PO_list

Cancel on ibuy

By item code

In [ ]:
driver = webdriver.Chrome("C:/Users/ywx610506/Documents/chromedriver.exe")
driver.implicitly_wait(10)
log_in('ibuy',"Yifan")
time.sleep(2)
#direct to PO cancel
driver.find_element_by_css_selector('[id="arrowIconSpan"]').click()
driver.find_element_by_css_selector("#MenuBar > li:nth-child(3) > a > span").click()
driver.find_element_by_css_selector("#MenuBar > li:nth-child(3) > ul > li:nth-child(1) > a > span").click()
driver.find_element_by_css_selector("#MenuBar > li:nth-child(3) > ul > li:nth-child(1) > ul > li:nth-child(2) > a > span").click()
driver.find_element_by_css_selector("#MenuBar > li:nth-child(3) > ul > li:nth-child(1) > ul > li:nth-child(2) > ul > li:nth-child(1) > a > span").click()
time.sleep(2)
driver.find_element_by_css_selector('#aCode').send_keys('5520911')
driver.find_element_by_css_selector('#aCode').send_keys(Keys.ENTER)
time.sleep(1)
driver.find_element_by_css_selector("#id_content_div > form > table.input_data_table > tbody > tr:nth-child(2) > td:nth-child(6) > nobr > input.button").click()
time.sleep(1)

for PO_number in PO_list:
    Row_count = Select(driver.find_element_by_name('pagination.rowsPerPage'))
    Row_count.select_by_value("50")
    time.sleep(3)
    driver.find_element_by_css_selector('[name="prChangeListVO.poNo"]').send_keys(PO_number)
    driver.find_element_by_css_selector('[name="prChangeListVO.poNo"]').send_keys(Keys.ENTER)
    time.sleep(2)
    Page_index=driver.find_element_by_css_selector('[class="t_page"]').text
    Page_total=Page_index.split('/')[1]
    for page in range(1,int(Page_total)+1):
        driver.find_element_by_css_selector('[name="inputPageNo"]').clear()
        driver.find_element_by_css_selector('[name="inputPageNo"]').send_keys(page)
        Go=driver.find_element_by_css_selector('[class="turn_page"]').find_element_by_css_selector('[class="button"]')
        Go.click()
        time.sleep(2)
        Select_table=driver.find_element_by_css_selector('[class="input_data_table"]')
        list_tr_row=Select_table.find_elements(By.TAG_NAME, "tr")
        for line in range(0,len(list_tr_row)-1):
            DU_loc='[name="prChangeListLine['+str(line)+'].siteId"]'
            item_code_loc='[name="prChangeListLine['+str(line)+'].item"]'
            DU_ID=list_tr_row[line+1].find_element_by_css_selector(DU_loc).get_attribute("value")
            item_code=list_tr_row[line+1].find_element_by_css_selector(item_code_loc).get_attribute("value")
            DU_list=list(set(list(Cancel_table.loc[PO_number].index.get_level_values('Site Code'))))
            if str(DU_ID) in DU_list:
                item_list=list(set(list(Cancel_table.loc[PO_number].loc[DU_ID].index.get_level_values('Item'))))
                if str(item_code) in item_list:
                    list_tr_row[line+1].find_element_by_css_selector('[name="prCheck"]').click()
                    line_date=pd.DataFrame([[PO_number,DU_ID,item_code]],columns=["PO","DU_ID","item"])
                    Check_table=Check_table.append(line_date, ignore_index=True)
    time.sleep(1)
    driver.find_element_by_css_selector('[name="btn2prChange"]').click()
    while True:
        try:
            driver.find_element_by_name("all").click()
            break
        except: 
            time.sleep(3)
    driver.find_element_by_css_selector("#changeCanbnt").click()
    time.sleep(1)
    driver.find_element_by_css_selector("#div1 > form > div > div > center > input").click()
    #select cancel reason
    Line_cancel_reason = Select(driver.find_element_by_name('prInfoVO.changeReason1'))
    Line_cancel_reason.select_by_value(Cancel_select)
    driver.find_element_by_name("prCancelRemark").send_keys(Cancel_reason)
    time.sleep(1)
    #select approver
    Approver = Select(driver.find_element_by_name('tempParamsMap(wfApprove)'))
    Approver.select_by_value("266055")
    time.sleep(1)
    driver.find_element_by_css_selector('[id="changeSubbnt"]').click()
    time.sleep(1)
    try:
        alert = driver.switch_to.alert
        pause=input('Please remove accepted item and press enter to continue')
        driver.find_element_by_css_selector('[id="changeSubbnt"]').click()
        alert.accept()
    except:
        None
print("All done")

By item description

In [ ]:
driver = webdriver.Chrome("C:/Users/ywx610506/Documents/chromedriver.exe")
driver.implicitly_wait(10)
log_in('ibuy',"Yifan")
time.sleep(2)
#direct to PO cancel
driver.find_element_by_css_selector('[id="arrowIconSpan"]').click()
driver.find_element_by_css_selector("#MenuBar > li:nth-child(3) > a > span").click()
driver.find_element_by_css_selector("#MenuBar > li:nth-child(3) > ul > li:nth-child(1) > a > span").click()
driver.find_element_by_css_selector("#MenuBar > li:nth-child(3) > ul > li:nth-child(1) > ul > li:nth-child(2) > a > span").click()
driver.find_element_by_css_selector("#MenuBar > li:nth-child(3) > ul > li:nth-child(1) > ul > li:nth-child(2) > ul > li:nth-child(1) > a > span").click()
time.sleep(2)
driver.find_element_by_css_selector('#aCode').send_keys('5520911')
driver.find_element_by_css_selector('#aCode').send_keys(Keys.ENTER)
time.sleep(1)
driver.find_element_by_css_selector("#id_content_div > form > table.input_data_table > tbody > tr:nth-child(2) > td:nth-child(6) > nobr > input.button").click()
time.sleep(1)

for PO_number in PO_list:
    Row_count = Select(driver.find_element_by_name('pagination.rowsPerPage'))
    Row_count.select_by_value("50")
    time.sleep(3)
    driver.find_element_by_css_selector('[name="prChangeListVO.poNo"]').send_keys(PO_number)
    driver.find_element_by_css_selector('[name="prChangeListVO.poNo"]').send_keys(Keys.ENTER)
    time.sleep(2)
    Page_index=driver.find_element_by_css_selector('[class="t_page"]').text
    Page_total=Page_index.split('/')[1]
    for page in range(1,int(Page_total)+1):
        driver.find_element_by_css_selector('[name="inputPageNo"]').clear()
        driver.find_element_by_css_selector('[name="inputPageNo"]').send_keys(page)
        Go=driver.find_element_by_css_selector('[class="turn_page"]').find_element_by_css_selector('[class="button"]')
        Go.click()
        time.sleep(2)
        Select_table=driver.find_element_by_css_selector('[class="input_data_table"]')
        list_tr_row=Select_table.find_elements(By.TAG_NAME, "tr")
        for line in range(0,len(list_tr_row)-1):
            DU_loc='[name="prChangeListLine['+str(line)+'].siteId"]'
            item_name_loc='[name="prChangeListLine['+str(line)+'].description"]'
            DU_ID=list_tr_row[line+1].find_element_by_css_selector(DU_loc).get_attribute("value")
            item_name=list_tr_row[line+1].find_element_by_css_selector(item_name_loc).get_attribute("value")
            DU_list=list(set(list(Cancel_table.loc[PO_number].index.get_level_values('Site ID'))))
            if str(DU_ID) in DU_list:
                item_list=list(set(list(Cancel_table.loc[PO_number].loc[DU_ID].index.get_level_values('PO Line Description'))))
                if str(item_name) in item_list:
                    list_tr_row[line+1].find_element_by_css_selector('[name="prCheck"]').click()
                    line_date=pd.DataFrame([[PO_number,DU_ID,item_name]],columns=["PO","DU_ID","item"])
                    Check_table=Check_table.append(line_date, ignore_index=True)
    time.sleep(1)
    driver.find_element_by_css_selector('[name="btn2prChange"]').click()
    while True:
        try:
            driver.find_element_by_name("all").click()
            break
        except: 
            time.sleep(3)
    driver.find_element_by_css_selector("#changeCanbnt").click()
    time.sleep(1)
    driver.find_element_by_css_selector("#div1 > form > div > div > center > input").click()
    #select cancel reason
    Line_cancel_reason = Select(driver.find_element_by_name('prInfoVO.changeReason1'))
    Line_cancel_reason.select_by_value(Cancel_select)
    driver.find_element_by_name("prCancelRemark").send_keys(Cancel_reason)   
    time.sleep(1)
    #select approver
    Approver = Select(driver.find_element_by_name('tempParamsMap(wfApprove)'))
    Approver.select_by_value("266055")
    time.sleep(1)
    driver.find_element_by_css_selector('[id="changeSubbnt"]').click()
    time.sleep(1)
    try:
        alert = driver.switch_to.alert
        pause=input('Please remove accepted item and press enter to continue')
    except:
        None
print("All done")

In [ ]:
writer = pd.ExcelWriter(r'D:\02.PO_cancellation\Process\Check_process.xlsx')
Check_table.to_excel(writer,'Sheet1')
writer.save()